## Motivation

Understanding the results of the STP is not straightforward and requires a grasp of the module equation. To simplify this, I will attempt to demonstrate the results using basic operations, eliminating the need to read the module equation.